In [1]:
#import modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
%matplotlib inline
import scipy as sp
import statsmodels.formula.api as smf

<h3>Functions to generate SFS</h3>

In [ ]:
#Function to get site counts for sfs
def getSiteCounts(df):
    Ln = df.Ln.sum()
    Ls = df.Ls.sum()
    L = Ln + Ls
    dN = (df.dN*df.Ln).sum()
    dS = (df.dS*df.Ls).sum()
    return([Ln,Ls,L,dN,dS])

In [ ]:
#Function generated sfs for Grapes-friendly format (folded SFS)
def getSFS(df, chromosomes, AFcol, consequence):
    #Get counts of alleles
    sfs = df.minor_alleles.value_counts()
    #Convert index from float to int
    sfs.index=sfs.index.astype(int)
    #Convert sfs to dataframe and sort index
    sfs=pd.DataFrame(sfs).sort_index()
    #Create list of zeroes
    sfs2=pd.DataFrame([0 for x in range(1,662)])
    #Shift index so that it starts from 0, not 1
    sfs2.index=[x+1 for x in sfs2.index]
    #Rename column to enable addition of dfs
    sfs2=sfs2.rename(columns={0:'minor_alleles'})
    sfs=sfs2+sfs
    #Convert NAs to 0
    sfs=sfs.fillna(0)
    #Convert floats to ints
    sfs=sfs.astype(int)
    return(sfs)

In [ ]:
#Main function to create sfs for Grapes and output to file
#Input is dataframe of snps; no. of chromosomes in sample; column containing allele frequency info; outFile
def getSFS_master(df, chromosomes, AFcol, outFile=''):    
    #Remove fixed or lost mutations (ie with AF=0 or AF=1)
    df = df[(df[AFcol]>0) & (df[AFcol]<(chromosomes))]
    #Convert AFs to MAFs
    df[AFcol] = np.where(df[AFcol]>(chromosomes/2), (chromosomes)-df[AFcol], df[AFcol])
    #get site counts (sitesCol is column with si)
    sites=getSiteCounts(df)
    #Get SFS for non-synonymous & synonymous sites
    sfsN=getSFS(df, chromosomes, AFcol, 'missense')
    sfsS=getSFS(df, chromosomes, AFcol, 'synonymous')
    Nnames = []
    lst=[]     
    #Add gene name to start of list
    lst.insert(0,'all_genes')
    #Add chromosome number to list
    lst.insert(1,chromosomes)
    #Add number of divergent sites and dN etc to end of list
    lst.append(int(np.round(sites[0],0)))
    lst.extend(list(sfsN.minor_alleles))
    lst.append(int(np.round(sites[1],0)))
    lst.extend(list(sfsS.minor_alleles))
    lst.append(int(np.round(sites[0],0)))
    lst.append(int(np.round(sites[3],0)))

    lst.append(int(np.round(sites[1],0)))
    lst.append(int(np.round(sites[4],0)))
    Nnames.append('gene')
    Nnames.append('chromosomes')
    Nnames.append('Ln(poly)')
    Nnames.extend(['Pn'+str(i) for i in range(1,len(sfsN)+1)])
    Nnames.append('Ls(poly)')
    Nnames.extend(['Ps'+str(i) for i in range(1,len(sfsS)+1)])
    Nnames.append('Ln(div)')
    Nnames.append('Dn')
    Nnames.append('Ls(div)')
    Nnames.append('Ds')

    dictionary = dict(zip(Nnames, lst))
    sfs=pd.DataFrame.from_dict(dictionary, orient='index').T
    #If outFile is empty, return sfs, otherwise output to file
    if(outFile!=''):
        sfs.to_csv(outFile, header=True, sep="\t", index=False)
    else:
        return(sfs)

<h3>Plotting grapes results</h3>

In [2]:
#Function takes in 2 grapes output files - main results file and second file for omega results
def get_results(resFile, omegaFile, mergeCol):
    rdf = pd.read_csv(resFile, sep='\t', index_col=False, header=0)

    rdf2 = pd.read_csv(omegaFile, sep=' ', names=['RSA_bin', 'omegaNA', 'CIs'])
    
    #Parse omega file for omegaNA results
    rdf2['CIs'] = rdf2.CIs.str.strip('[')
    rdf2['CIs'] = rdf2.CIs.str.strip(']')
    rdf2['CIs'] = rdf2['CIs'].astype('str')
    rdf2['omegaNA_low'] = [float(x[0]) for x in rdf2.CIs.str.split(',')]
    rdf2['omegaNA_high'] = [float(x[1]) for x in rdf2.CIs.str.split(',')]
    #merge dfs on mergeCol
    rdf = pd.merge(rdf, rdf2, on=mergeCol, how='inner')
    #sort values by mergeCol
    rdf = rdf.sort_values(mergeCol)
    return(rdf)

In [3]:
#Plot results from output of get_results function
def plot_results(rdf, ind_variable):
    sns.set(rc={'figure.figsize':(16,10),'axes.facecolor':'white','axes.edgecolor': '0.1'},font_scale=1.5)
    #Weighted regression for omegaA
    rdf['weight']=1/(((rdf.omegaA_high-rdf.omegaA_low)/4)**2)
    wmod = smf.wls("omegaA ~ " + ind_variable, data=rdf, weights = rdf.weight ).fit()
    
    g = sns.FacetGrid(rdf, height = 8)
    g = g.map(plt.scatter, ind_variable, "omegaA", edgecolor="w", color='b')
    #Generate label based on significance of results
    if(np.round(wmod.f_pvalue, 3) <= 0.001):
        l = r'$\omega_{a}$ (***)'
    elif(np.round(wmod.f_pvalue, 3) <= 0.01):
        l = r'$\omega_{a}$ (**)'   
    elif(np.round(wmod.f_pvalue, 3) <= 0.05):
        l = r'$\omega_{a}$ (*)' 
    elif(np.round(wmod.f_pvalue, 3) <= 0.1):
        l = r'$\omega_{a}$ (.)' 
    else:
        l = r'$\omega_{a}$' 
    plt.plot(rdf[ind_variable], wmod.fittedvalues, color='b', label=l)

    #Repeat for omegaNA
    rdf['weight']=1/(((rdf.omegaNA_high-rdf.omegaNA_low)/4)**2)
    wmod = smf.wls("omegaNA ~ " + ind_variable, data=rdf, weights = rdf.weight ).fit()
    g = g.map(plt.scatter, ind_variable, "omegaNA", edgecolor="w", color='r')
    #Generate label based on significance of results
    if(np.round(wmod.f_pvalue, 3) <= 0.001):
        l = r'$\omega_{na}$ (***)'
    elif(np.round(wmod.f_pvalue, 3) <= 0.01):
        l = r'$\omega_{na}$ (**)'   
    elif(np.round(wmod.f_pvalue, 3) <= 0.05):
        l = r'$\omega_{na}$ (*)' 
    elif(np.round(wmod.f_pvalue, 3) <= 0.1):
        l = r'$\omega_{na}$ (.)' 
    else:
        l = r'$\omega_{na}$' 
    plt.plot(rdf[ind_variable], wmod.fittedvalues, color='r', label=l)
    plt.xlabel(ind_variable)
    plt.ylabel(r'rate of evolution')
    plt.legend()
    plt.show() 
